# SQL #3 - SubQuery
Before start this notebook, please make sure to complete this following list:
- Install [XAMPP](https://www.apachefriends.org/index.html)
- Start MySQL Database, and Apache Webserver
- Install Package `SQLAlchemy` on notebook, this package is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL
- Import database `mysqlsampledatabase.sql` to MySQL Database

## Connect to database
Connect to localhost, using `classicmodels` database with default user. 

In [1]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

host = 'localhost'
port = "3306"
user = 'root'
password = ''
database = 'classicmodels'

#create connection to database
engine = create_engine('mysql+pymysql://'+user+':'+password+'@'+host+':'+port+'/'+database)
# engine = create_engine('mysql+pymysql://root: @localhost:3306/classicmodels)

def run(sql):
    df = pd.read_sql_query(sql,engine)
    return df

## Input CSV to MySQL
- Open `student.csv` file, see the structure of that file which contains `ID, Name` in their header.
- Read Data from `studen.csv` file
- input csv to SQL database

In [2]:
#CSV to DataFrame to MySQL
df_fromcsv = pd.read_csv('student.csv')
df_fromcsv.to_sql(name = 'student', con = engine, if_exists = 'replace', index = False)
# if_exists = 'fail' | 'replace' | 'append'

In [3]:
df_result = run("SELECT * FROM student")
print(df_result)

   ID    Name
0  11     Ani
1  22    Budi
2  33   Citra
3  44   Damar
4  55    Esti


## Add data using Query INSERT
Insert will be autocommit

In [4]:
engine.execute("INSERT INTO student VALUES(66,'Fajar')")

In [5]:
run("SELECT * FROM student")

,ID,Name
0,11,Ani
1,22,Budi
2,33,Citra
3,44,Damar
4,55,Esti
5,66,Fajar


## Fetch Data

In [6]:
engine.execute("select * from student").fetchall()

[(11, ' Ani'),
 (22, ' Budi'),
 (33, ' Citra'),
 (44, ' Damar'),
 (55, ' Esti'),
 (66, 'Fajar')]

## Read Data in the Tabel Form

In [7]:
df_results = run("select * from student")
print(df_results)

   ID    Name
0  11     Ani
1  22    Budi
2  33   Citra
3  44   Damar
4  55    Esti
5  66   Fajar


# Subquery

A **subquery** is a query that is nested inside a SELECT , INSERT , UPDATE , or DELETE statement, or inside another subquery. A subquery can be used anywhere an expression is allowed.

<img src="https://www.mysqltutorial.org/wp-content/uploads/2013/02/mysql-subquery.gif">


In [8]:
run("""
SELECT 
    lastName, 
    firstName
FROM
    employees
WHERE
    officeCode IN (SELECT 
            officeCode
        FROM
            offices
        WHERE
            country = 'USA')
""")

,lastName,firstName
0,Murphy,Diane
1,Patterson,Mary
2,Firrelli,Jeff
3,Bow,Anthony
4,Jennings,Leslie
5,Thompson,Leslie
6,Firrelli,Julie
7,Patterson,Steve
8,Tseng,Foon Yue
9,Vanauf,George


## Comparison
<img src="http://www.mysqltutorial.org/wp-content/uploads/2013/05/payments_table.png">

In [9]:
run("""
SELECT 
    customerNumber, checkNumber, amount
FROM
    payments
WHERE
    amount = (SELECT 
            MAX(amount)
        FROM
            payments);
""")

,customerNumber,checkNumber,amount
0,141,JE105477,120166.58


In [10]:
run("""
SELECT 
    customerNumber, checkNumber, amount
FROM
    payments
WHERE
    amount > (SELECT 
            AVG(amount)
        FROM
            payments);

""")

,customerNumber,checkNumber,amount
0,112,HQ55022,32641.98
1,112,ND748579,33347.88
2,114,GG31455,45864.03
3,114,MA765515,82261.22
4,114,NR27552,44894.74
5,119,LN373447,47924.19
6,119,NG94694,49523.67
7,121,DB889831,50218.95
8,121,MA302151,34638.14
9,124,AE215433,101244.59


## Using the IN and NOT IN Operators

In [11]:
run("""
SELECT 
    customerName
FROM
    customers
WHERE
    customerNumber NOT IN (SELECT DISTINCT
            customerNumber
        FROM
            orders);

""")

,customerName
0,Havel & Zbyszek Co
1,American Souvenirs Inc
2,Porto Imports Co.
3,"Asian Shopping Network, Co"
4,Natürlich Autos
5,ANG Resellers
6,Messner Shopping Network
7,"Franken Gifts, Co"
8,BG&E Collectables
9,Schuyler Imports


## Subquery in FROM
The goal is to create a temporary table that does not have tables in the database

In [12]:
run("""
SELECT 
    MAX(items), MIN(items), FLOOR(AVG(items))
FROM
    (SELECT 
        orderNumber, COUNT(orderNumber) AS items
    FROM
        orderdetails
    GROUP BY orderNumber) AS lineitems;

""")

,MAX(items),MIN(items),FLOOR(AVG(items))
0,18,1,9.0
